In [2]:
import pandas as pd
RC = pd.read_csv('RC_icmecat.csv')

Time conversions

In [ ]:
# Remove trailing annotation like (A), (W), W, S, etc. from the 'Disturbance Y/M/D (UT)' column
RC['Disturbance Y/M/D (UT)'] = RC['Disturbance Y/M/D (UT)'].str.replace(r"[^\d?/]+$", "", regex=True)

# Remove any non-numeric characters (like 'S') from 'dV (km/s)' column
RC['dV (km/s)'] = RC['dV (km/s)'].str.extract(r"(\d+(?:\.\d+)?)")[0]
RC['Dst (nT)'] = RC['Dst (nT)'].str.extract(r"(\d+(?:\.\d+)?)")[0]

# Convert the first three columns to datetime and remove timezone info if present
RC_dates = RC.iloc[:, :3].apply(pd.to_datetime, errors='coerce').apply(lambda x: x.dt.tz_convert(None) if x.dt.tz else x)

RC_dates = RC_dates.rename(columns={RC_dates.columns[0]: 'Disturbance_time', RC_dates.columns[1]: 'ICME_Start', RC_dates.columns[2]: 'ICME_End'})

# add columns 10 to 14 from RC to the new DataFrame
RC_dates = pd.concat([RC_dates, RC.iloc[:, 10:14].reset_index(drop=True)], axis=1)
RC_dates = pd.concat([RC_dates, RC.iloc[:, 15].reset_index(drop=True)], axis=1)

Resize to time range of GFOC

In [12]:
GFOC_start = pd.to_datetime('2023-01-03 12:00:00')
GFOC_end = pd.to_datetime('2024-06-30 11:59:30')

# only select times that are in the range of the GFOC data
RC_dates = RC_dates[pd.to_datetime(RC_dates['ICME_End'], format='%Y-%m-%d %H:%M:%S') >= GFOC_start]
RC_dates = RC_dates[pd.to_datetime(RC_dates['Disturbance_time'], format='%Y-%m-%d %H:%M:%S') <= GFOC_end]

# save the modified DataFrame to a new CSV file
RC_dates.to_csv('RC_icmecat_GFOC.csv', index=False)